In [ ]:
%load_ext autoreload
%autoreload 2

import requests


from pow.service.client import PowClient
from pow.models.utils import PARAMS_V1, PARAMS_V2

URL3080 = "http://localhost:8082"
URLH100 = "http://89.169.111.65:8080"
RECEIVER_URL = "http://89.169.111.65:8181"
BLOCK_HASH = "some-shared-hash"
PUBLIC_KEY = "public_key_1"
R_TARGET = 1.4013564660458173
BATCH_SIZE = 100

params = PARAMS_V2
client3080 = PowClient(URL3080)
clientH100 = PowClient(URLH100)

client3080.status()
clientH100.status()

client3080.stop()
clientH100.stop()


In [ ]:
def get_proof_batches(url=RECEIVER_URL):
    response = requests.get(f"{url}/generated")
    if response.status_code == 200:
        return response.json()["proof_batches"]
    raise Exception(f"Error: {response.status_code} - {response.text}")

def get_val_proof_batches(url=RECEIVER_URL):
    response = requests.get(f"{url}/validated")
    if response.status_code == 200:
        return response.json()["validated_batches"]
    raise Exception(f"Error: {response.status_code} - {response.text}")

In [31]:
client3080.stop()
client3080.init_generate(
    node_id=0,
    node_count=1,
    url=RECEIVER_URL,
    block_hash=BLOCK_HASH,
    block_height=0,
    public_key=PUBLIC_KEY,
    batch_size=BATCH_SIZE,
    r_target=1.4013564660458173,
    fraud_threshold=1e-7,
    params=params
)
client3080.start_validation()


{'status': 'OK',
 'pow_status': {'status': 'VALIDATING', 'is_model_initialized': False}}

In [ ]:
clientH100.stop()
clientH100.init_generate(
    node_id=0,
    node_count=1,
    url=RECEIVER_URL,
    block_hash=BLOCK_HASH,
    block_height=0,
    public_key=PUBLIC_KEY+"asxasd",
    batch_size=BATCH_SIZE,
    r_target=1.4013564660458173,
    fraud_threshold=0.01,
    params=params
)

# clientH100.start_validation()

In [27]:
from pow.compute.compute import ProofBatch

generated = get_proof_batches(RECEIVER_URL)
proof_batches = [ProofBatch(**batch) for batch in get_proof_batches(RECEIVER_URL)]

In [32]:
for batch in proof_batches:
    client3080.validate(batch)

In [49]:
validated_batches = get_val_proof_batches(RECEIVER_URL)

In [51]:
for v in validated_batches:
    pass

In [58]:
v["fraud_threshold"]

1e-07

In [59]:
v

{'public_key': 'public_key_1asxasd',
 'block_hash': 'some-shared-hash',
 'block_height': 0,
 'nonces': [77,
  79,
  83,
  109,
  113,
  119,
  125,
  141,
  169,
  197,
  216,
  226,
  236,
  259,
  263,
  272,
  273,
  274,
  289,
  290,
  298,
  302,
  303,
  358,
  441,
  444,
  449,
  460,
  461,
  476,
  527,
  542,
  549,
  553,
  556,
  573,
  594,
  611,
  628,
  643,
  651,
  684,
  710,
  722,
  726,
  728,
  735,
  742,
  757,
  781,
  787,
  788,
  832,
  840,
  858,
  862,
  865,
  878,
  879,
  891,
  894,
  899,
  983,
  990,
  997,
  1002,
  1020,
  1027,
  1049,
  1053,
  1061,
  1087,
  1094,
  1119,
  1123,
  1127,
  1130,
  1191,
  1221,
  1273,
  1286,
  1307,
  1315,
  1321,
  1329,
  1374,
  1388,
  1394,
  1443,
  1469,
  1472,
  1474,
  1478,
  1489,
  1502,
  1539,
  1608,
  1634,
  1652,
  1657,
  1691,
  1698,
  1700,
  1717,
  1721,
  1740,
  1768,
  1797,
  1808,
  1831,
  1837,
  1849,
  1856,
  1873,
  1958,
  1985,
  2001,
  2010,
  2021,
  2026,
  2051

In [ ]:
[x for x in v["dist"] if x > v["r_target"]]

[1.401356479254226]

In [50]:
for v in validated_batches:
    print(len(v["nonces"]), v["probability_honest"])

18 1.0
15 1.0
19 1.0
21 1.0
13 1.0
12 1.0
16 1.0
11 1.0
18 1.0
19 1.0
16 1.0
18 1.0
10 1.0
13 1.0
14 1.0
11 1.0
15 1.0
13 1.0
13 1.0
15 1.0
13 1.0
14 1.0
16 1.0
16 1.0
10 1.0
16 1.0
13 1.0
16 1.0
12 1.0
15 1.0
21 1.0
15 1.0
11 1.0
11 1.0
18 1.0
11 1.0
19 1.0
9 1.0
24 1.0
21 1.0
10 1.0
9 1.0
21 1.0
18 1.0
8 1.0
11 1.0
9 1.0
15 1.0
15 1.0
11 1.0
13 1.0
12 1.0
8 1.0
11 1.0
19 1.0
8 1.0
12 1.0
19 1.0
8 1.0
11 1.0
12 1.0
13 1.0
13 1.0
12 1.0
11 0.005486270604389431
16 1.0
15 1.0
14 1.0
19 1.0
20 1.0
9 1.0
17 1.0
11 1.0
14 1.0
15 1.0
12 1.0
5 1.0
14 1.0
18 1.3726425170898442e-55
1091 0.42052712244150825


In [38]:
proof_batches[0]

copy_proof_batches = ProofBatch(**generated[0])

In [43]:
copy_proof_batches.nonces = [x + 8 for x in copy_proof_batches.nonces]
client3080.validate(copy_proof_batches)

In [ ]:
for v in validated_batches:
    print(len(v["nonces"]), v["probability_honest"])

In [46]:
merged_proof = ProofBatch.merge(proof_batches)
client3080.validate(merged_proof)

In [ ]:
for v in validated_batches:
    print(len(v["nonces"]), v["probability_honest"])